# Analyse de la température moyenne mensuelle et son lissage sur 60 mois


L'analyse présentée dans ce notebook se concentre sur l'étude de la température moyenne mensuelle et de son lissage sur 60 mois à partir d'un jeu de données météorologiques exhaustif. Les données utilisées proviennent du site data.gouv.fr et sont accessibles à l'URL : https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz. Ce fichier contient des enregistrements quotidiens de diverses variables météorologiques tels que les précipitations, la température minimale, maximale et moyenne, ainsi que d'autres informations pertinentes collectées à partir de plusieurs postes météorologiques en France.


L'objectif de cette analyse est de calculer la température moyenne mensuelle et d'appliquer un lissage sur 60 mois pour mettre en évidence les tendances à long terme et les variations saisonnières. Nous utiliserons une requête SQL spécifique pour traiter les données et une visualisation graphique pour représenter les résultats de manière claire et interactive.


## Méthodologie


La méthodologie adoptée pour cette analyse comporte deux étapes principales : le traitement des données via une requête SQL et la visualisation des résultats à l'aide d'une bibliothèque graphique. La requête SQL est conçue pour calculer la moyenne mensuelle de la température moyenne (TM) et pour appliquer un lissage sur 60 mois à ces moyennes mensuelles. Les résultats sont ensuite exportés vers un dataframe Pandas qui est utilisé pour créer une visualisation interactive avec Bokeh, représentant la température moyenne mensuelle brute et la série lissée sur 60 mois.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH 
-- Calcul de la moyenne mensuelle de TM
monthly_avg_tm AS (
    SELECT 
        CAST(SUBSTRING("AAAAMMJJ", 1, 6) AS INTEGER) AS yyyymm,
        AVG(CAST("TM" AS DOUBLE)) AS avg_tm
    FROM 
        loaded_dataset
    WHERE 
        "TM" IS NOT NULL AND "TM" != 'None'
    GROUP BY 
        yyyymm
),
-- Calcul du lissage sur 60 mois de la moyenne mensuelle de TM
smoothed_avg_tm AS (
    SELECT 
        yyyymm,
        avg_tm,
        AVG(avg_tm) OVER (ORDER BY yyyymm ROWS BETWEEN 29 PRECEDING AND 30 FOLLOWING) AS smoothed_tm
    FROM 
        monthly_avg_tm
)
SELECT 
    yyyymm,
    avg_tm AS tm_moyenne_mensuelle,
    smoothed_tm AS tm_lissage_60_mois
FROM 
    smoothed_avg_tm
ORDER BY 
    yyyymm """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 888 lignes


,yyyymm,tm_moyenne_mensuelle,tm_lissage_60_mois
0,195001,6.309677,13.249556
1,195002,10.043636,13.460054
2,195003,9.653226,13.513486
3,195004,10.645000,13.552179
4,195005,15.209836,13.458640


## 📈 Visualisation

La bibliothèque principale utilisée est Bokeh, qui permet de créer des visualisations interactives et élégantes. Bokeh est adaptée pour représenter des données temporelles comme ici, avec des fonctionnalités telles que le zoom et les infobulles interactives. Cela permet une exploration détaillée et intuitive des données.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, LinearAxis, DatetimeTickFormatter
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral4
from datetime import datetime
import numpy as np

# Conversion des colonnes yyyymm en datetime
df['date'] = pd.to_datetime(df['yyyymm'].astype(str), format='%Y%m')

# Préparation des données
source = ColumnDataSource(data=dict(
    date=df['date'],
    tm_moyenne_mensuelle=df['tm_moyenne_mensuelle'],
    tm_lissage_60_mois=df['tm_lissage_60_mois'],
    yyyymm=df['yyyymm'].astype(str)
))

# Création de la figure avec un design élégant
dataviz = figure(
    title="Évolution de la température moyenne mensuelle et lissage sur 60 mois",
    width=1000, height=500,
    x_axis_label=None,
    y_axis_label='Température (°C)',
    tools="pan,box_zoom,wheel_zoom,reset",
    toolbar_location="above",
    sizing_mode="stretch_width"
)

# Configuration de l'axe Y avec des limites appropriées
dataviz.y_range.start = df['tm_moyenne_mensuelle'].min() - 2
dataviz.y_range.end = df['tm_moyenne_mensuelle'].max() + 2

# Ajout des courbes avec des styles distinctifs
dataviz.line(
    x='date', y='tm_moyenne_mensuelle',
    source=source,
    line_color='#BDBDBD',
    line_width=1,
    alpha=0.7,
    legend_label='Température brute'
)

dataviz.line(
    x='date', y='tm_lissage_60_mois',
    source=source,
    line_color='#E53E3E',
    line_width=2,
    alpha=0.9,
    legend_label='Lissage sur 60 mois'
)

# Configuration de l'axe X pour afficher les dates de manière lisible
dataviz.xaxis.formatter = DatetimeTickFormatter(
    years="%Y",
    months="%m/%Y"
)
dataviz.xaxis.axis_label_text_font_size = "12pt"
dataviz.yaxis.axis_label_text_font_size = "12pt"

# Infobulle interactive et détaillée
hover = HoverTool(
    tooltips=[
        ('Date', '@yyyymm'),
        ('Température brute', '@tm_moyenne_mensuelle{0.00} °C'),
        ('Lissage 60 mois', '@tm_lissage_60_mois{0.00} °C')
    ],
    mode='vline',
    line_policy='nearest',
    point_policy='snap_to_data'
)
dataviz.add_tools(hover)

# Stylisation du titre et de la légende
dataviz.title.text_font_size = "14pt"
dataviz.title.text_font = "Helvetica Neue"
dataviz.legend.location = "top_left"
dataviz.legend.label_text_font_size = "11pt"
dataviz.legend.background_fill_alpha = 0.8
dataviz.legend.border_line_color = None

# Ajout de la grille avec des styles subtils
dataviz.grid.grid_line_alpha = 0.3
dataviz.grid.grid_line_color = '#CCCCCC'

# Fond transparent
dataviz.background_fill_color = None
dataviz.border_fill_color = None
dataviz

figure(id='p1006', ...)

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250801_211423.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250801_211423.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Bokeh. Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250801_211423.html
--> Création du marqueur de capture d'écran.


C:\Users\ami\AppData\Local\Temp\ipykernel_25296\429089486.py:67: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
